<a href="https://colab.research.google.com/github/omar1slam/Computer-Vision/blob/master/Image_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import imageio as im
from PIL import Image
from imageio import imread
from google.colab import drive
import matplotlib.pyplot as plt
from sklearn import svm
import cv2
drive.mount("/content/drive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
img = im.imread(r'/content/drive/My Drive/TestingImages/108073.jpg')
img_cluster = Image.open('/content/drive/My Drive/TestingImages/187039.jpg')
img = np.ndarray.astype(img, np.float32) / 255.0
height, width, channels = img.shape

Contrast stretching:


In [ ]:
def normalizeRed(intensity):
    iI  = intensity
    minI  = 86
    maxI = 230
    minO= 0
    maxO= 255
    iO  = (iI-minI)*(((maxO-minO)/(maxI-minI))+minO)
    return iO
    
def normalizeGreen(intensity):
    iI  = intensity
    minI = 90
    maxI = 225
    minO = 0
    maxO = 255
    iO = (iI-minI)*(((maxO-minO)/(maxI-minI))+minO)

    return iO

def normalizeBlue(intensity):
    iI = intensity
    minI= 100
    maxI= 210
    minO= 0
    maxO= 255
    iO  = (iI-minI)*(((maxO-minO)/(maxI-minI))+minO)
    return iO

def normalizeImage(image):
  Bands  = image.split()
  normRed  = Bands[0].point(normalizeRed)
  normGreen = Bands[1].point(normalizeGreen)
  normBlue = Bands[2].point(normalizeBlue)
  return Image.merge("RGB", (normRed, normGreen, normBlue))

# **C-Means Clustering:**

In [ ]:
def C_Means(img):
  h,w,m = np.shape(img)
  seg_img = np.zeros((h,w,3),np.int8)
  objmean=np.array([50,100,200])
  bgmean=np.array([0,50,40])
  temp_obj_mean = np.array([1,1,1])
  temp_bg_mean = np.array([1,1,1])
  iterations = 0


  while(1):
    BGred = list()
    BGblue = list()
    BGgreen = list()
    Objred = list()
    Objblue = list()
    Objgreen = list()
    for i in range(h):
      for j in range(w):
        I = np.array([img[i,j,0],img[i,j,1],img[i,j,2]])
        if np.sqrt((I-objmean).dot(I-objmean)) < np.sqrt((I-bgmean).dot(I-bgmean)):
          Objred.append(img[i,j,0])
          Objblue.append(img[i,j,1])
          Objgreen.append(img[i,j,2])

        else:
          BGred.append(img[i,j,0])
          BGblue.append(img[i,j,1])
          BGgreen.append(img[i,j,2])

    iterations += 1
    temp_obj_mean[0] = sum(Objred)/len(Objred)
    temp_obj_mean[1] = sum(Objgreen)/len(Objgreen)
    temp_obj_mean[2] = sum(Objblue)/len(Objblue)
    temp_bg_mean[0] = sum(BGred)/len(BGred)
    temp_bg_mean[1] = sum(BGgreen)/len(BGgreen)
    temp_bg_mean[2] = sum(BGblue)/len(BGblue)
    print(temp_obj_mean,'.............obj.............',objmean)
    print(temp_bg_mean,'...............bg...........',bgmean)

    if (temp_obj_mean[0] == objmean[0] and temp_obj_mean[1] == objmean[1] and temp_obj_mean[2] == objmean[2] and temp_bg_mean[0] == bgmean[0] and temp_bg_mean[1] == bgmean[1] and temp_bg_mean[2] == bgmean[2]  ):
      
      print(5)
      break

    else:
      print(1)
      objmean = np.copy(temp_obj_mean)
      bgmean = np.copy(temp_bg_mean)

    if iterations == 20:
      break

  for k in range(h):
    for n in range(w):
      I = np.array([img[k,n,0],img[k,n,1],img[k,n,2]])
      if np.sqrt((I-objmean).dot(I-objmean)) < np.sqrt((I-bgmean).dot(I-bgmean)):
        seg_img[k,n,0] = objmean[0]
        seg_img[k,n,1] = objmean[1]
        seg_img[k,n,2] = objmean[2]
      else:
        seg_img[k,n,0] = bgmean[0]
        seg_img[k,n,1] = bgmean[1]
        seg_img[k,n,2] = bgmean[2]

  return seg_img



# **Nearest Neighbor:**

In [ ]:
def nearest_neighbor(image, obj_batch_orig, bg_batch_orig):

    result = np.zeros((image.shape[0], image.shape[1]))
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            obj_batch = obj_batch_orig
            bg_batch = bg_batch_orig
            while True:
                obj_diff = abs(image[i, j] - obj_batch)
                bg_diff = abs(image[i, j] - bg_batch)

                obj_diff_norm = np.linalg.norm(obj_diff, axis=-1)
                bg_diff_norm = np.linalg.norm(bg_diff, axis=-1)

                obj_min_diff = np.amin(obj_diff_norm)
                bg_min_diff = np.amin(bg_diff_norm)

                obj_min_index = np.argmin(obj_diff_norm)
                bg_min_index = np.argmin(bg_diff_norm)

                if obj_min_diff < bg_min_diff:
                    result[i, j] = 0
                    break
                elif obj_min_diff > bg_min_diff:
                    result[i, j] = 1
                    break
                else:
                    obj_batch = np.delete(obj_batch, obj_min_index, 0)
                    bg_batch = np.delete(bg_batch, bg_min_index, 0)

    return result

# **K-Nearest Neighbor:**

In [ ]:
def k_nearest_neighbor(image, K, obj_batch_orig, bg_batch_orig):

    result = np.zeros((image.shape[0], image.shape[1]))

    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            obj_counter = 0
            bg_counter = 0
            obj_batch = obj_batch_orig
            bg_batch = bg_batch_orig
            for k in range(K):
                while True:
                    obj_diff = abs(image[i, j] - obj_batch)
                    bg_diff = abs(image[i, j] - bg_batch)

                    obj_diff_norm = np.linalg.norm(obj_diff, axis=-1)
                    bg_diff_norm = np.linalg.norm(bg_diff, axis=-1)

                    obj_min_diff = np.amin(obj_diff_norm)
                    bg_min_diff = np.amin(bg_diff_norm)

                    obj_min_index = np.argmin(obj_diff_norm)
                    bg_min_index = np.argmin(bg_diff_norm)

                    if obj_min_diff < bg_min_diff:
                        obj_counter += 1
                        obj_batch = np.delete(obj_batch, obj_min_index, 0)
                        break
                    elif obj_min_diff > bg_min_diff:
                        bg_counter += 1
                        bg_batch = np.delete(bg_batch, bg_min_index, 0)
                        break
                    else:
                        obj_batch = np.delete(obj_batch, obj_min_index, 0)
                        bg_batch = np.delete(bg_batch, bg_min_index, 0)

            if obj_counter > bg_counter:
                result[i, j] = 0
            else:
                result[i, j] = 1

    return result

In [ ]:
def shuffle(features, labels):
    permutation = list(np.random.permutation(features.shape[0]))
    features_shuffled = features[permutation, :]
    shuffled_labels = labels[permutation]

    return features_shuffled, shuffled_labels

In [ ]:
def get_areas(two_batches=False):

    if two_batches:

        x, y, w, h = np.array(cv2.selectROI("Image", img, False))
        obj_batch = img[y:y+h, x:x+w]
        x, y, w, h = np.array(cv2.selectROI("Image", img, False))
        bg_batch = img[y:y+h, x:x+w]
        x, y, w, h = np.array(cv2.selectROI("Image", img, False))
        obj_batch2 = img[y:y+h, x:x+w]
        x, y, w, h = np.array(cv2.selectROI("Image", img, False))
        bg_batch2 = img[y:y+h, x:x+w]

        obj_batch = np.reshape(obj_batch, (obj_batch.shape[0] * obj_batch.shape[1], 3))
        bg_batch = np.reshape(bg_batch, (bg_batch.shape[0] * bg_batch.shape[1], 3))
        obj_batch2 = np.reshape(obj_batch2, (obj_batch2.shape[0] * obj_batch2.shape[1], 3))
        bg_batch2 = np.reshape(bg_batch2, (bg_batch2.shape[0] * bg_batch2.shape[1], 3))

        obj = np.concatenate((obj_batch, obj_batch2), axis=0)
        bg = np.concatenate((bg_batch, bg_batch2), axis=0)
        features = np.concatenate((obj, bg), axis=0)

        obj_labels = np.full(obj_batch.shape[0], 0)
        bg_labels = np.full(bg_batch.shape[0], 1)
        obj_labels2 = np.full(obj_batch2.shape[0], 0)
        bg_labels2 = np.full(bg_batch2.shape[0], 1)
        labels = np.concatenate((obj_labels, obj_labels2, bg_labels, bg_labels2))

        features, labels = shuffle(features, labels)

        return obj, bg, features, labels

    else:
        x, y, w, h = np.array(cv2.selectROI("Image", img, False))
        obj_batch = img[y:y+h, x:x+w]
        x, y, w, h = np.array(cv2.selectROI("Image", img, False))
        bg_batch = img[y:y+h, x:x+w]

        obj_batch = np.reshape(obj_batch, (obj_batch.shape[0] * obj_batch.shape[1], 3))
        bg_batch = np.reshape(bg_batch, (bg_batch.shape[0] * bg_batch.shape[1], 3))

        obj_labels = np.full(obj_batch.shape[0], 0)
        bg_labels = np.full(bg_batch.shape[0], 1)

        features = np.concatenate((obj_batch, bg_batch), axis=0)
        labels = np.concatenate((obj_labels, bg_labels))

        features, labels = shuffle(features, labels)

        return obj_batch, bg_batch, features, labels

In [ ]:
def get_points():
    obj_batch = np.array([[img[226, 127]]
                             , [img[149, 331]]
                             , [img[188, 424]]
                             , [img[192, 208]]
                             , [img[167, 408]]
                             , [img[101, 48]]
                             , [img[81, 305]]
                             , [img[203, 314]]
                             , [img[141, 428]]
                             , [img[100, 241]]])
    obj_batch = obj_batch.reshape((obj_batch.shape[0], 3))

    bg_batch = np.array([[img[35, 450]]
                            , [img[150, 463]]
                            , [img[309, 388]]
                            , [img[195, 352]]
                            , [img[306, 200]]
                            , [img[178, 364]]
                            , [img[310, 74]]
                            , [img[244, 462]]
                            , [img[296, 204]]])
    bg_batch = bg_batch.reshape((bg_batch.shape[0], 3))

    features = np.concatenate((obj_batch, bg_batch), axis=0)

    obj_labels = np.full(obj_batch.shape[0], 0)
    bg_labels = np.full(bg_batch.shape[0], 1)
    labels = np.concatenate((obj_labels, bg_labels))

    features, labels = shuffle(features, labels)

    return obj_batch, bg_batch, features, labels

#**Support Vector Machine:**

In [ ]:
def support_vector_machine(image, X, Y):
    model = svm.SVC(kernel='linear')
    model.fit(X, Y)
    height, width = image.shape[0], image.shape[1]
    for i in range(height):
        for j in range(width):
            pixel = np.reshape(image[i, j], (1, 3))
            image[i, j] = model.predict(pixel)
    return image

# **Bayes' Classification:**

In [ ]:
def Mean(batch):
  x = np.array([np.mean(batch[:,:,0]),np.mean(batch[:,:,1]),np.mean(batch[:,:,2])])
  return x

def Cov(batch):
  h,w,m = np.shape(batch)
  mean = Mean(batch)
  x = np.zeros((3,3), dtype=float)
  for i in range(h):
    for j in range(w):
       x += np.matmul((batch[i][j] - mean).reshape(3,1),((batch[i][j]-mean).reshape(3,1)).transpose())
  x = (1/(h*w))*x
  return x

def probability(pixel,batch,cov,mean):
  c = np.asmatrix(cov)
  m = mean
  I = (pixel - m).reshape(3,1)
  #print(I)
  const = 1/(np.sqrt(2*3.14159)*(np.linalg.det(c))**1.5)
  exp = np.exp(-0.5*(I.transpose()*np.linalg.inv(c)*I))
  expp = exp[0,0]
  return const*expp

# **Main**

In [ ]:
cov_obj = Cov(obj_b)
cov_bg = Cov(bg_b)
m_obj = Mean(obj_b)
m_bg = Mean(bg_b)
h,w,m = np.shape(img2)
Seg_img_bayes = np.zeros((h,w),np.int8)

In [ ]:
print(np.shape(img))
imgn = normalizeImage(img_cluster)
Img_arr= np.array(imgn)
Seg_Img = C_Means(Img_arr)


In [ ]:
Seg_Img = Image.fromarray((Seg_Img * 255).astype(np.uint8))
plt.figure(figsize=(10,10))
plt.imshow(Seg_Img)


In [ ]:
obj_batch, bg_batch, features, labels = get_areas(True)

In [ ]:
img = nearest_neighbor(img, obj_batch, bg_batch)
plt.imshow(img)

In [ ]:
img = k_nearest_neighbor(img, 3, obj_batch, bg_batch)
plt.imshow(img)

In [ ]:
img = support_vector_machine(img, features, labels)
plt.imshow(img)

In [ ]:
for i in range(h):
  for j in range(w):
    if (0.77*(probability(img2[i][j],obj_b,cov_obj,m_obj))> 0.23*(probability(img2[i][j],bg_b,cov_bg,m_bg))):
      Seg_img_bayes[i][j]=1
    else:
      Seg_img_bayes[i][j]=0

plt.imshow(Seg_img_bayes)
print(Seg_img_bayes)

In [ ]:
from scipy import ndimage
result = ndimage.median_filter(Seg_Img, size=3)

plt.figure(figsize=(10,10))
plt.imshow(result)